# Data Preparation 

In [85]:
# load packages

import numpy as np
import pandas as pd

In [33]:
# load dataset 

Chicago_Crimes_2005_to_2007 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2005_to_2007.csv", usecols = range(23))
Chicago_Crimes_2008_to_2011 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2008_to_2011.csv", usecols = range(23))
Chicago_Crimes_2012_to_2017 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2012_to_2017.csv", usecols = range(23))

# merge data 
Chicago_Crimes_2005_to_2017 = pd.concat([Chicago_Crimes_2005_to_2007, Chicago_Crimes_2008_to_2011, Chicago_Crimes_2012_to_2017])

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
# filter the dataset by years and export data from 2007 to 2017

Chicago_Crimes_2005_to_2017.head(20)
print(Chicago_Crimes_2005_to_2017.Year.unique())
Chicago_Crimes_2007_to_2017 = Chicago_Crimes_2005_to_2017.loc[(Chicago_Crimes_2005_to_2017['Year'] >= 2007) 
                                                              & (Chicago_Crimes_2005_to_2017['Year'] <= 2017)]
print(Chicago_Crimes_2007_to_2017.Year.unique())

# store the new DataFrame to a csv file for later use.
Chicago_Crimes_2007_to_2017.to_csv("Chicago_Crimes_2007_to_2017.csv",index=False)

[2006 2005 2007 2008 2009 2011 2010 2016 2015 2012 2014 2013 2017]
[2007 2008 2009 2011 2010 2016 2015 2012 2014 2013 2017]


In [127]:
# load data from 2007 to 2017
df = pd.read_csv("Chicago_Crimes_2007_to_2017.csv")

# check for features 
print(df.columns.values)

# drop unnecessary features, col 0
df.drop(df.columns[0], axis = 1, inplace = True)
print(df.dtypes)
print(df.shape) # (4496569, 22), 22 features 

# removing unneeded features
df.drop(['X Coordinate', 'Y Coordinate', 'Updated On', 'Latitude', 'Longitude', 'Location'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Unnamed: 0' 'ID' 'Case Number' 'Date' 'Block' 'IUCR' 'Primary Type'
 'Description' 'Location Description' 'Arrest' 'Domestic' 'Beat'
 'District' 'Ward' 'Community Area' 'FBI Code' 'X Coordinate'
 'Y Coordinate' 'Year' 'Updated On' 'Latitude' 'Longitude' 'Location']
ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude                object
Location                 object
dtype: object
(4496569, 22)


In [128]:
# check uniqueness of ID
df['ID'].is_unique #no unique, cases being recorded multiple times 

False

In [129]:
# return only one record per case by removing duplicated rows 
# ex: df[df['ID'] == 7781132]; 7815319, 7781132, 6023200  

print(len(df.ID.unique())) #3182854, but we have 4496569 rows 
df.drop_duplicates(keep = 'first', inplace = True) #remove duplicates 

# drop Case number, we are using ID as the primary key
df.drop(['Case Number'], axis = 1, inplace = True)
df.reset_index(inplace = True)

print(df.shape) #(3182854, 16) #matched 

3182854
(3182854, 16)


In [130]:
# check for missing values

df.isna().any() # Case Number, Location Description, District, Ward, Community Area

# Replace null values with 'Other' option; categorical 
df['Location Description'].value_counts() # 154 types + other
df['Location Description'].isna().sum() # 1933 null values 
df['Location Description'].fillna("Other", inplace = True)

# Numerical missing values 
df['District'].value_counts() # 25districts + nan
df['District'].isna().sum() # 43 nan

df['Ward'].value_counts() # 50 + nan
df['Ward'].isna().sum() # 56 nan

df['Community Area'].value_counts() # 78 + nan
df['Community Area'].isna().sum() # 926 nan

############TBC

926

In [131]:
# separate Date column into two col: date and time 

df.Date.head(10)
datetime = pd.to_datetime(df.Date, errors='coerce')

df.drop(['Date'], axis = 1, inplace = True)
df['Date'] = datetime.dt.date
df['Time'] = datetime.dt.time

df.dtypes

index                     int64
ID                        int64
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
Year                      int64
Date                     object
Time                     object
dtype: object

In [133]:
df.to_csv("cleanedData_ver1.csv",index=False)

In [132]:
df

,index,ID,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Year,Date,Time
0,0,5223506,028XX S DR MARTIN LUTHER KING JR DR,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,2122,2.0,4.0,35.0,26,2007,2007-01-03,10:00:00
1,1,5224416,009XX W HURON ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,1323,12.0,27.0,24.0,14,2007,2007-01-04,18:00:00
2,2,5224520,004XX W ADAMS ST,5002,OTHER OFFENSE,OTHER VEHICLE OFFENSE,STREET,True,False,111,1.0,2.0,28.0,26,2007,2007-01-05,09:30:00
3,3,5225556,005XX W MADISON ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,111,1.0,42.0,28.0,14,2007,2007-01-05,19:00:00
4,4,5227933,043XX N LAKE SHORE DR NB,2024,NARCOTICS,POSS: HEROIN(WHITE),STREET,True,False,2323,19.0,46.0,3.0,18,2007,2007-01-04,14:50:00
5,5,5227797,001XX N LEAMINGTON AVE,2094,NARCOTICS,ATTEMPT POSSESSION CANNABIS,SIDEWALK,True,False,1532,15.0,28.0,25.0,18,2007,2007-01-05,11:35:00
6,6,5242118,015XX W HOWARD ST,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,True,False,2422,24.0,49.0,1.0,18,2007,2007-01-07,13:00:00
7,7,5242417,019XX W 69TH ST,460,BATTERY,SIMPLE,SIDEWALK,False,False,726,7.0,17.0,67.0,08B,2007,2007-01-13,01:39:00
8,8,5243014,003XX W 40TH ST,460,BATTERY,SIMPLE,FACTORY/MANUFACTURING BUILDING,False,False,925,9.0,11.0,61.0,08B,2007,2007-01-13,09:00:00
9,9,5243687,003XX W 40TH ST,850,THEFT,ATTEMPT THEFT,FACTORY/MANUFACTURING BUILDING,False,False,925,9.0,11.0,61.0,6,2007,2007-01-14,02:50:00
